In [12]:
import os
from tqdm import tqdm
from glob import glob

import numpy as np
import pandas as pd

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def calculate_moments(image):
    mean = np.mean(image)
    std_dev = np.std(image)
    skewness = np.mean((image - mean) ** 3) / (std_dev ** 3)
    kurtosis = np.mean((image - mean) ** 4) / (std_dev ** 4) - 3
    
    return mean, std_dev, skewness, kurtosis

In [3]:
def calculate_color_moments(image, color_space):
    converted_image = cv2.cvtColor(image, color_space)
    components_moments = [calculate_moments(converted_image[:, :, i]) for i in range(converted_image.shape[2])]
    color_moments = [moment for component_moments in components_moments for moment in component_moments]
    
    return color_moments

In [4]:
def create_ring_masks(image, num_rings):
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    masks = []

    for i in range(num_rings):
        radius_inner = int((i / num_rings) * min(height, width))
        radius_outer = int(((i + 1) / num_rings) * min(height, width))

        mask = np.zeros_like(image, dtype=np.uint8)
        cv2.circle(mask, center, radius_outer, (255, 255, 255), thickness = -1)
        cv2.circle(mask, center, radius_inner, (0, 0, 0), thickness = -1)

        masks.append(mask)

    return masks

In [5]:
def calculate_edge_gradients(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    gradient_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize = 5)
    gradient_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize = 5)
    
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    gradient_orientation = np.arctan2(gradient_y, gradient_x)

    return gradient_magnitude, gradient_orientation

In [6]:
def calculate_shape_features(contour):
    moments = cv2.moments(contour)
    area = moments['m00']
    perimeter = cv2.arcLength(contour, True)

    aspect_ratio = moments['mu20'] / moments['mu02']
    circularity = (4 * np.pi * area) / (perimeter**2)

    return aspect_ratio, circularity

In [7]:
def analyze_image_circles(image, num_rings, color_space):
    color_moments = calculate_color_moments(image, color_space)
    ring_masks = create_ring_masks(image, num_rings)
    
    results = []

    for i, ring_mask in enumerate(ring_masks):
        ring_image = cv2.bitwise_and(image, ring_mask)

        gradient_magnitude, gradient_orientation = calculate_edge_gradients(ring_image)

        contours, _ = cv2.findContours(np.uint8(gradient_magnitude), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        ring_results = []

        for contour in contours:
            aspect_ratio, circularity = calculate_shape_features(contour)
            ring_results.append((aspect_ratio, circularity))

        results.append(ring_results)

    return color_moments, results

In [8]:
# Image directories
nickels_cropped_dir = "./nickels_cropped/"
nickels_cropped_files = glob(f"{nickels_cropped_dir}/*")

quarters_cropped_dir = "./quarters_cropped/"
quarters_cropped_files = glob(f"{quarters_cropped_dir}/*")

cents_cropped_dir = "./cents_cropped/"
cents_cropped_files = glob(f"{cents_cropped_dir}/*")

In [9]:
image = cv2.imread(nickels_cropped_files[0])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

num_rings = 5
color_space = cv2.COLOR_RGB2HSV


color_moments, ring_results = analyze_image_circles(image, num_rings, color_space)

print("Color Moments:", color_moments)
print("Shape Features at Rings:", ring_results)

Color Moments: [34.91298203920938, 49.4254019802513, 1.5360416418172016, 1.1031716843711035, 36.77967729542835, 48.77613073558815, 2.485138370837458, 7.707695747368616, 62.179383061285876, 66.4903971244285, 1.179582474615523, 0.8460686182002997]
Shape Features at Rings: [[(1.0, 0.8972445777756118)], [(0.9999890205039326, 0.8921082715132201)], [(0.9999999999454303, 0.7853981633974483), (0.9999999999776482, 0.7853981633974483), (1.44, 0.7789072694850726), (0.680527691309478, 0.7809989786265679), (1.0134478432846508, 0.5660399847178756), (1.9544310013227837, 0.7908057336749165), (1.954431001309575, 0.7908057336749165), (2.25, 0.7539822368615503)], [], []]


In [22]:
for item in ring_results:
    print(item)

[(1.0, 0.8919455973714382)]
[(0.9999091866438361, 0.8931398692541477)]
[(1.0000000000143583, 0.8168387726383408), (1.0013427847967276, 0.5454276693755952)]
[]
[]


In [13]:
color_moments_list = []
ring_results_list = []
img_types = []

directories = [nickels_cropped_dir, quarters_cropped_dir, cents_cropped_dir]

for directory in directories:
    if "nickels" in directory:
        img_type = "Nickels"
    elif "quarters" in directory:
        img_type = "Quarters"
    elif "cents" in directory:
        img_type = "Cents"

    cropped_files = glob(f"{directory}/*")

    for file in tqdm(cropped_files, desc = f"Processing {img_type}"):
        img = cv2.imread(file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        color_moments, ring_results = analyze_image_circles(img, num_rings = 5, color_space = cv2.COLOR_RGB2HSV)

        color_moments_list.append(color_moments)
        ring_results_list.append(ring_results)
        img_types.append(img_type)

Processing Cents: 100%|██████████| 1161/1161 [01:55<00:00, 10.05it/s]


In [17]:
color_moments_df = pd.DataFrame(color_moments_list, columns = [f"moment_{i}" for i in range(len(color_moments_list[0]))])
color_moments_df

,moment_0,moment_1,moment_2,moment_3,moment_4,moment_5,moment_6,moment_7,moment_8,moment_9,moment_10,moment_11
0,34.912982,49.425402,1.536042,1.103172,36.779677,48.776131,2.485138,7.707696,62.179383,66.490397,1.179582,0.846069
1,15.192815,20.212510,4.750719,27.640190,119.280534,77.324156,-0.340664,-1.033499,90.463499,72.630385,0.418324,-0.693313
2,18.222754,18.947552,3.778876,23.678416,77.671726,61.022187,0.363980,-0.057632,59.674764,53.441376,0.948860,1.080395
3,13.941320,16.093284,4.565348,31.444495,92.949518,69.776854,-0.137195,-0.918098,64.038939,51.467725,0.051462,-0.850299
4,15.664649,17.991799,4.742764,29.897838,130.610471,83.050445,-0.702247,-1.050464,68.707073,53.554933,0.303945,-0.386744
...,...,...,...,...,...,...,...,...,...,...,...,...
3088,16.477225,30.579675,3.887326,15.226098,88.121363,66.045105,0.576766,0.349155,77.687150,86.190561,0.872139,-0.831526
3089,12.458681,21.919505,4.808776,25.011056,144.300706,80.667891,-0.868848,-0.471002,94.001319,79.469272,0.477822,-0.896944
3090,15.980744,26.327165,4.312295,19.945097,108.864506,67.737117,-0.091134,-0.289864,87.882094,90.150138,0.618810,-1.318374
3091,13.529356,21.011548,4.582628,23.197718,135.285037,76.750237,-0.807639,-0.469346,115.565462,85.491435,-0.014386,-1.160932


In [21]:
ring_results_list

[[[(1.0, 0.8972445777756118)],
  [(0.9999890205039326, 0.8921082715132201)],
  [(0.9999999999454303, 0.7853981633974483),
   (0.9999999999776482, 0.7853981633974483),
   (1.44, 0.7789072694850726),
   (0.680527691309478, 0.7809989786265679),
   (1.0134478432846508, 0.5660399847178756),
   (1.9544310013227837, 0.7908057336749165),
   (1.954431001309575, 0.7908057336749165),
   (2.25, 0.7539822368615503)],
  [],
  []],
 [[(1.0, 0.8951917951388424)],
  [(1.0000255440165795, 0.8950558584048408)],
  [(1.5625000000032743, 0.7757018897752576),
   (0.199460278337102, 0.6720981925216009),
   (0.20030234315948614, 0.5345164612311879),
   (1.5624999999942148, 0.7757018897752576),
   (0.9999999999940453, 0.7853981633974483),
   (1.0002084978987276, 0.5564638137802516)],
  [],
  []],
 [[(0.9999623940019963, 0.8964506859328828)],
  [(0.9999838298194703, 0.8950280365328959)],
  [(1.44, 0.7789072694850726),
   (1.0000000000223517, 0.7853981633974483),
   (0.5116578683610741, 0.7908057336749165),
   (1

In [14]:
img_types_df = pd.DataFrame({"img_type": img_types})
final_df = pd.concat([img_types_df, color_moments_df, ring_results_df], axis = 1)

ValueError: 2 columns passed, passed data had 37 columns